# Winners Analysis - Council of Alphas

Pipeline: 3 strategies/family (12 total) - 1 champion/family (4 total) - 3 hybrids - 2D regime filter optimizer - ranked survivors

Portfolio simulation: $100k initial capital, 0.5% risk per trade, ATR-based position sizing (TBM 2.0/1.0/24, 1h candles, 0.075% fee).

Dynamically loads whatever ranked results exist from `data/results/ranked/`.

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

INITIAL_CAPITAL = 100_000
RANKED_DIR = Path('../data/results/ranked')

# Load ranked summary to discover survivors dynamically
with open(RANKED_DIR / 'ranked_summary.json') as f:
    ranked_summary = json.load(f)

print(f'Survivors: {len(ranked_summary)}')
for s in ranked_summary:
    print(f'  #{s["rank"]}: {s["name"]} (fitness={s["fitness"]:.3f}, sharpe={s["sharpe"]:.4f}, trades={s["trades"]})')

# Load trade logs dynamically
state = pd.read_parquet('../data/state_matrix_1h.parquet')
long_outcomes = state['tbm_long_outcome'].astype(str).to_numpy(copy=False)
short_outcomes = state['tbm_short_outcome'].astype(str).to_numpy(copy=False)

def attach_tbm_outcome(df):
    out = df.copy()
    entries = out['entry_index'].to_numpy(dtype=np.int64)
    sides = out['side'].to_numpy(dtype=np.int8)
    outcomes = np.full(len(out), 'TIMEOUT', dtype=object)
    long_mask = sides == 1
    short_mask = sides == -1
    outcomes[long_mask] = long_outcomes[entries[long_mask]]
    outcomes[short_mask] = short_outcomes[entries[short_mask]]
    out['tbm_outcome'] = outcomes
    return out

palette = ['#00CC96', '#636EFA', '#EF553B', '#AB63FA', '#FFA15A']

strategies = {}
colors = {}
for i, s in enumerate(ranked_summary):
    rank = s['rank']
    name = s['name']
    label = f'{name.replace("_", " ").title()} (#{rank})'
    trade_log_path = RANKED_DIR / f'{rank}_{name}' / 'trade_log.csv'
    df = pd.read_csv(trade_log_path, parse_dates=['entry_ts', 'exit_ts'])
    strategies[label] = attach_tbm_outcome(df)
    colors[label] = palette[i % len(palette)]

print(f'\nLoaded {len(strategies)} strategies')

Survivors: 3
  #1: consensus_gate (fitness=1.037, sharpe=0.1669, trades=499)
  #2: regime_router (fitness=0.120, sharpe=0.0323, trades=1102)
  #3: weighted_combination (fitness=-0.159, sharpe=-0.0533, trades=4832)

Loaded 3 strategies


## 1. Head - First Trades

In [2]:
for name, df in strategies.items():
    print(f'=== {name} ({len(df):,} trades) ===')
    display(df.head(10))
    print()

=== Consensus Gate (#1) (499 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-05 19:00:00+00:00,2022-01-05 21:00:00+00:00,115,117,2,-1,0.009676,100967.578338,NY,DOWNTREND,HIGH_VOL,TP
1,2022-01-06 14:00:00+00:00,2022-01-06 17:00:00+00:00,134,137,3,-1,-0.005168,100445.789609,NY,DOWNTREND,HIGH_VOL,SL
2,2022-01-12 13:00:00+00:00,2022-01-12 16:00:00+00:00,277,280,3,1,0.009737,101423.832476,NY,UPTREND,HIGH_VOL,TP
3,2022-01-12 19:00:00+00:00,2022-01-13 00:00:00+00:00,283,288,5,1,-0.005276,100888.682768,NY,UPTREND,HIGH_VOL,SL
4,2022-01-21 14:00:00+00:00,2022-01-21 15:00:00+00:00,494,495,1,-1,-0.005136,100370.506143,NY,DOWNTREND,HIGH_VOL,SL
5,2022-01-31 22:00:00+00:00,2022-02-01 03:00:00+00:00,742,747,5,1,0.009789,101353.081214,OTHER,UPTREND,HIGH_VOL,TP
6,2022-02-01 14:00:00+00:00,2022-02-01 16:00:00+00:00,758,760,2,1,0.009837,102350.136388,NY,UPTREND,HIGH_VOL,TP
7,2022-02-01 20:00:00+00:00,2022-02-02 14:00:00+00:00,764,782,18,1,-0.005147,101823.316807,NY,UPTREND,HIGH_VOL,SL
8,2022-02-07 14:00:00+00:00,2022-02-07 15:00:00+00:00,902,903,1,1,-0.005250,101288.779207,NY,UPTREND,HIGH_VOL,SL
9,2022-02-07 17:00:00+00:00,2022-02-07 20:00:00+00:00,905,908,3,1,-0.005234,100758.622756,NY,UPTREND,HIGH_VOL,SL



=== Regime Router (#2) (1,102 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-02 00:00:00+00:00,2022-01-02 04:00:00+00:00,24,28,4,-1,0.009584,100958.377622,ASIA,CONSOLIDATION,LOW_VOL,TP
1,2022-01-02 06:00:00+00:00,2022-01-02 16:00:00+00:00,30,40,10,-1,-0.005419,100411.305071,ASIA,CONSOLIDATION,LOW_VOL,SL
2,2022-01-03 00:00:00+00:00,2022-01-03 03:00:00+00:00,48,51,3,-1,0.009539,101369.145319,ASIA,CONSOLIDATION,LOW_VOL,TP
3,2022-01-03 05:00:00+00:00,2022-01-03 15:00:00+00:00,53,63,10,1,-0.005465,100815.133943,ASIA,CONSOLIDATION,LOW_VOL,SL
4,2022-01-03 21:00:00+00:00,2022-01-03 22:00:00+00:00,69,70,1,-1,-0.005406,100270.134916,OTHER,DOWNTREND,HIGH_VOL,SL
5,2022-01-03 23:00:00+00:00,2022-01-04 04:00:00+00:00,71,76,5,-1,0.009603,101233.077417,OTHER,DOWNTREND,HIGH_VOL,TP
6,2022-01-05 21:00:00+00:00,2022-01-05 22:00:00+00:00,117,118,1,-1,0.009730,102218.046721,OTHER,DOWNTREND,HIGH_VOL,TP
7,2022-01-05 23:00:00+00:00,2022-01-06 03:00:00+00:00,119,123,4,-1,0.009780,103217.724008,OTHER,DOWNTREND,HIGH_VOL,TP
8,2022-01-07 23:00:00+00:00,2022-01-08 01:00:00+00:00,167,169,2,-1,-0.005141,102687.087411,OTHER,DOWNTREND,HIGH_VOL,SL
9,2022-01-10 22:00:00+00:00,2022-01-11 04:00:00+00:00,238,244,6,-1,-0.005192,102153.922404,OTHER,CONSOLIDATION,HIGH_VOL,SL



=== Weighted Combination (#3) (4,832 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-02 00:00:00+00:00,2022-01-02 03:00:00+00:00,24,27,3,1,-0.005416,99458.377622,ASIA,CONSOLIDATION,LOW_VOL,SL
1,2022-01-02 04:00:00+00:00,2022-01-03 01:00:00+00:00,28,49,21,1,-0.005393,98921.976803,ASIA,CONSOLIDATION,LOW_VOL,SL
2,2022-01-03 02:00:00+00:00,2022-01-03 04:00:00+00:00,50,52,2,1,-0.005449,98382.926328,ASIA,CONSOLIDATION,LOW_VOL,SL
3,2022-01-03 05:00:00+00:00,2022-01-03 11:00:00+00:00,53,59,6,-1,-0.005465,97845.235492,ASIA,CONSOLIDATION,LOW_VOL,SL
4,2022-01-03 13:00:00+00:00,2022-01-03 15:00:00+00:00,61,63,2,-1,0.009515,98776.226872,NY,UPTREND,LOW_VOL,TP
5,2022-01-03 16:00:00+00:00,2022-01-03 20:00:00+00:00,64,68,4,1,-0.005418,98241.030463,NY,CONSOLIDATION,HIGH_VOL,SL
6,2022-01-03 21:00:00+00:00,2022-01-03 22:00:00+00:00,69,70,1,-1,-0.005406,97709.946846,OTHER,DOWNTREND,HIGH_VOL,SL
7,2022-01-03 23:00:00+00:00,2022-01-04 00:00:00+00:00,71,72,1,1,-0.005397,97182.653423,OTHER,DOWNTREND,HIGH_VOL,SL
8,2022-01-04 01:00:00+00:00,2022-01-04 11:00:00+00:00,73,83,10,1,0.009623,98117.880548,ASIA,DOWNTREND,HIGH_VOL,TP
9,2022-01-04 14:00:00+00:00,2022-01-04 18:00:00+00:00,86,90,4,-1,0.009666,99066.312669,NY,DOWNTREND,HIGH_VOL,TP


## 2. Info - DataFrame Structure

In [3]:
for name, df in strategies.items():
    print(f'=== {name} ===')
    print(f'Shape: {df.shape}')
    print()
    df.info()
    print('\n')

=== Consensus Gate (#1) ===
Shape: (499, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   entry_ts          499 non-null    datetime64[ns, UTC]
 1   exit_ts           499 non-null    datetime64[ns, UTC]
 2   entry_index       499 non-null    int64              
 3   exit_index        499 non-null    int64              
 4   duration          499 non-null    int64              
 5   side              499 non-null    int64              
 6   net_trade_return  499 non-null    float64            
 7   account_balance   499 non-null    float64            
 8   session           499 non-null    object             
 9   trend_regime      499 non-null    object             
 10  vol_regime        499 non-null    object             
 11  tbm_outcome       499 non-null    object             
dtypes: datetime64[ns, 

## 3. Portfolio Performance

In [4]:
# net_trade_return is now the LEVERAGED portfolio return (not raw asset return)
# account_balance tracks compounding equity
for name, df in strategies.items():
    returns = df['net_trade_return']
    outcomes = df['tbm_outcome']
    final_equity = df['account_balance'].iloc[-1]
    total_return = (final_equity - INITIAL_CAPITAL) / INITIAL_CAPITAL
    
    # Compounded check (should match account_balance)
    comp_return = (1 + returns).prod() - 1
    
    # Win/loss based on TBM outcomes only (TP/SL). Timeouts excluded.
    tp_returns = returns[outcomes == 'TP']
    sl_returns = returns[outcomes == 'SL']
    timeout_count = int((outcomes == 'TIMEOUT').sum())
    wins = int(len(tp_returns))
    losses = int(len(sl_returns))
    scored = wins + losses
    wr = (wins / scored) if scored > 0 else np.nan
    avg_win = tp_returns.mean()
    avg_loss = sl_returns.mean()
    rr = avg_win / abs(avg_loss) if pd.notna(avg_loss) and avg_loss != 0 else np.nan
    
    # Drawdown
    peak = df['account_balance'].cummax()
    drawdown = (df['account_balance'] - peak) / peak
    max_dd = drawdown.min()
    
    print(f'=== {name} ===')
    print(f'  Total trades:       {len(df):,}')
    print(f'  Final equity:       ${final_equity:,.2f}')
    print(f'  Total return:       {total_return:+.2%}')
    print(f'  Max drawdown:       {max_dd:.2%}')
    print(f'  Win rate (TP/SL):   {wr:.1%}' if pd.notna(wr) else '  Win rate (TP/SL):   N/A')
    print(f'  TP hits / SL hits:  {wins:,} / {losses:,}')
    print(f'  Timeouts:           {timeout_count:,}')
    print(f'  Avg win (portfolio):{avg_win*100:>+10.4f}%')
    print(f'  Avg loss (portfolio):{avg_loss*100:>+9.4f}%')
    print(f'  Risk-reward:        {rr:.2f}')
    print()

=== Consensus Gate (#1) ===
  Total trades:       499
  Final equity:       $182,430.11
  Total return:       +82.43%
  Max drawdown:       -7.97%
  Win rate (TP/SL):   43.4%
  TP hits / SL hits:  213 / 278
  Timeouts:           8
  Avg win (portfolio):   +0.9713%
  Avg loss (portfolio):  -0.5294%
  Risk-reward:        1.83

=== Regime Router (#2) ===
  Total trades:       1,102
  Final equity:       $125,483.32
  Total return:       +25.48%
  Max drawdown:       -15.90%
  Win rate (TP/SL):   36.9%
  TP hits / SL hits:  399 / 682
  Timeouts:           21
  Avg win (portfolio):   +0.9685%
  Avg loss (portfolio):  -0.5317%
  Risk-reward:        1.82

=== Weighted Combination (#3) ===
  Total trades:       4,832
  Final equity:       $14,789.13
  Total return:       -85.21%
  Max drawdown:       -86.21%
  Win rate (TP/SL):   32.5%
  TP hits / SL hits:  1,534 / 3,185
  Timeouts:           113
  Avg win (portfolio):   +0.9690%
  Avg loss (portfolio):  -0.5309%
  Risk-reward:        1.83



## 4. Sharpe Ratio Analysis

In [5]:
# Load state matrix to compute per-trade leverage (needed for zero-fee estimate)
state = pd.read_parquet('../data/state_matrix_1h.parquet')
close_arr = state['close'].values
atr_arr = state['ATR_24'].values

FEE = 0.00075  # 0.075% taker fee
TBM_LOSS = 1.0
RISK_PCT = 0.005

for name, df in strategies.items():
    returns = df['net_trade_return']
    
    # -- Standard annualized Sharpe (daily resampled, sqrt(365) for crypto) --
    daily = df.set_index('entry_ts')['net_trade_return'].resample('1D').sum().fillna(0)
    ann_sharpe = (daily.mean() / daily.std()) * np.sqrt(365) if daily.std() > 0 else 0
    
    # Per-trade Sharpe (raw)
    mean_ret = returns.mean()
    std_ret = returns.std()
    per_trade_sharpe = mean_ret / std_ret if std_ret > 0 else 0
    
    # Zero-fee estimate: add back leverage * fee per trade
    entries = df['entry_index'].values
    sl_pct = TBM_LOSS * atr_arr[entries] / close_arr[entries]
    leverage = np.minimum(RISK_PCT / sl_pct, 20.0)
    gross_returns = returns + leverage * FEE
    
    gross_mean = gross_returns.mean()
    gross_std = gross_returns.std()
    gross_sharpe = gross_mean / gross_std if gross_std > 0 else 0
    
    print(f'=== {name} ===')
    print(f'  Annualized Sharpe:      {ann_sharpe:.4f}  (daily resampled, sqrt(365))')
    print(f'  Per-trade Sharpe:       {per_trade_sharpe:.4f}')
    print(f'  Zero-fee per-trade:     {gross_sharpe:.4f}')
    print(f'  Mean return per trade:  {mean_ret:.6f} ({mean_ret*100:.4f}%)')
    print(f'  Std dev of returns:     {std_ret:.6f} ({std_ret*100:.4f}%)')
    print(f'  Avg leverage:           {leverage.mean():.2f}x')
    print(f'  Skewness:               {returns.skew():.4f}')
    print(f'  Kurtosis:               {returns.kurtosis():.4f}')
    print()

=== Consensus Gate (#1) ===
  Annualized Sharpe:      1.7568  (daily resampled, sqrt(365))
  Per-trade Sharpe:       0.1667
  Zero-fee per-trade:     0.2061
  Mean return per trade:  0.001233 (0.1233%)
  Std dev of returns:     0.007394 (0.7394%)
  Avg leverage:           0.39x
  Skewness:               0.2635
  Kurtosis:               -1.9234

=== Regime Router (#2) ===
  Annualized Sharpe:      0.5401  (daily resampled, sqrt(365))
  Per-trade Sharpe:       0.0323
  Zero-fee per-trade:     0.0762
  Mean return per trade:  0.000232 (0.0232%)
  Std dev of returns:     0.007181 (0.7181%)
  Avg leverage:           0.42x
  Skewness:               0.5424
  Kurtosis:               -1.6894

=== Weighted Combination (#3) ===
  Annualized Sharpe:      -1.7597  (daily resampled, sqrt(365))
  Per-trade Sharpe:       -0.0533
  Zero-fee per-trade:     -0.0091
  Mean return per trade:  -0.000371 (-0.0371%)
  Std dev of returns:     0.006969 (0.6969%)
  Avg leverage:           0.41x
  Skewness:      

## 5. Dollar Equity Curve ($100k start, 0.5% risk, 0.075% fee)

In [6]:
fig = make_subplots(
    rows=len(strategies), cols=1,
    subplot_titles=list(strategies.keys()),
    vertical_spacing=0.08,
)

for i, (name, df) in enumerate(strategies.items(), 1):
    fig.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        ),
        row=i, col=1,
    )
    
    fig.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5, row=i, col=1)
    fig.update_yaxes(title_text='Equity ($)', tickformat='$,.0f', row=i, col=1)

fig.update_xaxes(title_text='Date', row=len(strategies), col=1)

fig.update_layout(
    height=300 * len(strategies),
    title_text='Dollar Equity Curve - $100k, 0.5% Risk Per Trade (TBM 2.0/1.0/24, 1h, 0.075% fee)',
    showlegend=True,
    template='plotly_white',
)

fig.show()

In [7]:
# Overlay comparison
fig2 = go.Figure()

for name, df in strategies.items():
    fig2.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        )
    )

fig2.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5)

fig2.update_layout(
    height=500,
    title_text='Equity Overlay - All Three Hybrids ($100k Start)',
    yaxis_title='Equity ($)',
    yaxis_tickformat='$,.0f',
    xaxis_title='Date',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
)

fig2.show()

## 6. Average Holding Time

In [8]:
# duration column is in bars (1h candles)
for name, df in strategies.items():
    holding_hours = df['duration']  # 1 bar = 1 hour on 1h data
    
    print(f'=== {name} ===')
    print(f'  Mean:   {holding_hours.mean():.2f} hours')
    print(f'  Median: {holding_hours.median():.2f} hours')
    print(f'  Min:    {holding_hours.min()} hours')
    print(f'  Max:    {holding_hours.max()} hours')
    print(f'  Hit time barrier (duration=24 bars / 24h): {(df["duration"] == 24).sum():,} ({(df["duration"] == 24).mean():.1%})')
    print()

=== Consensus Gate (#1) ===
  Mean:   5.00 hours
  Median: 3.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 9 (1.8%)

=== Regime Router (#2) ===
  Mean:   5.75 hours
  Median: 4.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 26 (2.4%)

=== Weighted Combination (#3) ===
  Mean:   5.97 hours
  Median: 4.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 132 (2.7%)

